# Import library et data

In [1]:
import folium
import json
import numpy as np
import plotly.express as px
from folium.plugins import HeatMap
from folium import Choropleth, LayerControl, GeoJsonTooltip
import pandas as pd


In [2]:
nb_voitures = pd.read_csv("data/nb_voiture_annee_cdr.csv")
nb_voiture_commune = pd.read_csv("data/nb_voiture_commune.csv")
nb_voiture_dep = pd.read_csv("data/nb_voiture_dep.csv")
nb_voiture_reg = pd.read_csv("data/nb_voiture_reg.csv")
bornes_vehicules_dep = pd.read_csv("data/croisement_donnee_borne_voiture_departement.csv", encoding="utf-8")
bornes_vehicules_reg = pd.read_csv("data/croisement_donnee_borne_voiture_region.csv", encoding="utf-8")

bornes = pd.read_csv("data/bornes_completes.csv")
bornes2 = pd.read_csv("data/Bornes_nettoye2.csv", delimiter = ";")
population2 = pd.read_csv("data/population2.csv")
trafic_dep = pd.read_csv("data/tmja_dep_df.csv")
trafic_reg = pd.read_csv("data/tmja_reg.csv")

with open("data/communes.geojson", 'r') as f:
    geojson_data_com = json.load(f)

with open("data/france_departments.geojson", 'r') as f:
    geojson_data_dep = json.load(f)
with open("data/carte_interactive_avec_bornes.html", "r"   ) as file:
    carte_html = file.read()
with open("data/carte_tmja_troncons.html", "r"   ) as file:
    carte_html2 = file.read()

with open("data/regions.geojson", 'r') as f:
    geojson_data_reg = json.load(f)

C:\Users\MRH4\AppData\Local\Temp\ipykernel_5480\3401793680.py:1: DtypeWarning: Columns (0,5) have mixed types. Specify dtype option on import or set low_memory=False.
  nb_voitures = pd.read_csv("data/nb_voiture_annee_cdr.csv")
C:\Users\MRH4\AppData\Local\Temp\ipykernel_5480\3401793680.py:8: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  bornes = pd.read_csv("data/bornes_completes.csv")
C:\Users\MRH4\AppData\Local\Temp\ipykernel_5480\3401793680.py:9: DtypeWarning: Columns (18,19,20,21,22,23,24,29,33) have mixed types. Specify dtype option on import or set low_memory=False.
  bornes2 = pd.read_csv("data/Bornes_nettoye2.csv", delimiter = ";")


In [3]:
def create_map(nb_voiture_commune_dep, geojson_data,col_granu, info_carte):
    # Initialisation de la carte centrée sur la France
    map = folium.Map(location=[46.603354, 1.8883344], zoom_start=6, tiles='CartoDB positron')

    if info_carte == "nombre de véhicule":
        # Conversion des données en dictionnaire pour relier avec le GeoJSON
        col = 'nb_vp_rechargeables_el'
    elif info_carte == "ratio de véhicule électrique par rapport au total":
        # Conversion des données en dictionnaire pour relier avec le GeoJSON
        col = 'ratio_elec_total'

    vehicle_dict = nb_voiture_commune_dep.set_index(col_granu)[col].to_dict()
    # Mets les keys au format list str
    vehicle_dict = {str(k): v for k, v in vehicle_dict.items()}
    # Ajout des données au GeoJSON
    for feature in geojson_data['features']:
        feature_code = feature['properties']['code']
        feature['properties']['vehicles'] = vehicle_dict.get(feature_code, 'N/A')

    # Ajout d'un choropleth pour afficher les données
    folium.Choropleth(
        geo_data=geojson_data,
        name="Véhicules électriques par département",
        data=nb_voiture_commune_dep,
        columns=[col_granu, col],
        key_on='feature.properties.code',
        fill_color='YlGnBu',
        fill_opacity=0.7,
        line_opacity=0.05,
        legend_name="Nombre de véhicules électriques"
    ).add_to(map)
    # Ajout de tooltips interactifs
    folium.GeoJson(
        geojson_data,
        name="Détails",
        tooltip=folium.GeoJsonTooltip(
            fields=['code', 'nom', 'vehicles'],
            aliases=['Code :', 'Nom:', 'Véhicules électriques:']
        )
    ).add_to(map)

    # Ajout des contrôles de couches
    folium.LayerControl().add_to(map)
    
    return map

In [5]:
geojson_data = geojson_data_com
info_carte = "nombre de véhicule"
selected_year = 2023
col_granu = "codgeo"
dataset = nb_voiture_commune
dataset = dataset[dataset["annee"] == selected_year]
dataset.drop(columns=["annee"], inplace=True)
dataset["ratio_elec_total"] = dataset["nb_vp_rechargeables_el"]	/ dataset["nb_vp"]
# Fait ca en pourcentage nb_voiture_commune_dep["ratio_elec_total"] avec 2 chiffres après la virgule
dataset["ratio_elec_total"] = dataset["ratio_elec_total"] * 100
dataset["ratio_elec_total"] = dataset["ratio_elec_total"].round(2)


map = create_map(dataset, geojson_data, col_granu, info_carte)

C:\Users\MRH4\AppData\Local\Temp\ipykernel_5480\224134718.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.drop(columns=["annee"], inplace=True)
C:\Users\MRH4\AppData\Local\Temp\ipykernel_5480\224134718.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["ratio_elec_total"] = dataset["nb_vp_rechargeables_el"]	/ dataset["nb_vp"]
C:\Users\MRH4\AppData\Local\Temp\ipykernel_5480\224134718.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [6]:
map.save("data/Carte_html/carte_commune_2023.html")